In [1]:
from pyspark.sql import SparkSession
from vars import *
from datetime import date
from functions import flatten_json, loadConfigs
from pyspark.sql.functions import lit

spark = SparkSession.builder \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider") \
    .config("spark.jars", "/jars/postgresql-42.2.5.jar") \
    .getOrCreate()
loadConfigs(spark.sparkContext)

from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [2]:
today = date.today().strftime('%Y%m%d')
today = 20230325
dateid = int(today)

In [3]:
df = spark.read.option("header", "true") \
    .parquet(f"s3a://{minio_bucket}/processed/popular_20230323.parquet")

In [88]:
df_source = spark.read.option("header", "true") \
    .parquet(f"s3a://{minio_bucket}/raw/popular_{today}.parquet")

In [1]:
# table_name = "staging.raw_data"
# mode = "append"
# properties = {"user": postgres_user, "password": postgres_pass}
# df.write.jdbc(url=jdbc_url, table=table_name, mode=mode, properties=properties)

In [12]:
sql_query = "SELECT * FROM staging.raw_data LIMIT 0"

df_target = spark.read.jdbc(url=postgres_url,
                     table="( {} ) as mytable".format(sql_query),
                     properties=postgres_props)

In [25]:
source_schema = df_source.schema
target_schema = df_target.schema

In [15]:
df1 = df_target

In [81]:
missing_columns = []
for field in source_schema.fields:
    if field.name not in [f.name for f in target_schema.fields]:
        col_name = field.name
        col_name = col_name.replace('>','_')
        missing_columns.append((col_name, field.dataType))
        

alter_table_command = "ALTER TABLE staging.raw_data\nADD COLUMN "
for col in missing_columns:
    alter_table_command += "{} {},\n".format(col[0], col[1])
    
alter_table_command = alter_table_command[:-2] + ";\n"
    

In [86]:
# print(alter_table_command)

In [84]:
# spark.sql(alter_table_command).write.jdbc(table="staging.raw_data", mode="append", properties=postgres_props)

In [85]:
# # execute the SQL command using JDBC
# spark.read.jdbc(url=postgres_url,
#                 table="( {} )".format(alter_table_command),
#                 properties=postgres_props)